# Weak Encryption Enabled and Kerberoast
Detects scenario where weak encryption is enabled for a user profile which could be used for hash/password cracking.

## Rule Content
```
- title: Weak Encryption Enabled and Kerberoast
  id: f6de9536-0441-4b3f-a646-f4e00f300ffd
  description: Detects scenario where weak encryption is enabled for a user profile
    which could be used for hash/password cracking.
  references:
  - https://adsecurity.org/?p=2053
  - https://www.harmj0y.net/blog/activedirectory/roasting-as-reps/
  author: '@neu5ron'
  tags:
  - attack.defense_evasion
  - attack.t1089
  logsource:
    product: windows
    service: security
    definition: 'Requirements: Audit Policy : Account Management > Audit User Account
      Management, Group Policy : Computer Configuration\Windows Settings\Security
      Settings\Advanced Audit Policy Configuration\Audit Policies\Account Management\Audit
      User Account Management'
    category: null
  detection:
    selection:
      EventID: 4738
    keywords:
      Message:
      - '*DES*'
      - '*Preauth*'
      - '*Encrypted*'
    filters:
      Message:
      - '*Enabled*'
    condition: selection and keywords and filters
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4738" AND Message.keyword:(*DES* OR *Preauth* OR *Encrypted*) AND Message.keyword:(*Enabled*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()